In [84]:
import re
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
cols=['标题', '单位', '时间', '报告地点:', '主办单位:']

In [117]:
class AnnouncementsData(object):
    '''
    获取info公告的数据
    '''
    COLUMNS = ['title', 'reporter', 'time', 'content', 'url']
    def __init__(self):
        self.root_url = "http://postinfo.tsinghua.edu.cn"
        self.enter_url_eduadmin = "/f/bangongtongzhi/more"
        self.enter_url_important = "/f/zhongyaogonggao/more"
        self.columns = ['title', 'reporter', 'time', 'content', 'url']
    
    def get_eduadmin_announcement_data(self, page_count:int=-1):
        """
        读取教务公告数据
        Parameter:
            page_count: int, 读取的页数 
        Return:
            pd.DataFrame
        """
        page = requests.get(self.root_url + self.enter_url_eduadmin)
        data_list = []
        count = 0
        while True:
            page_bs = BeautifulSoup(page.text, 'lxml')
            lis_page = page_bs.find_all(name='li')
            for li in lis_page:
                if li.find('em') is None:
                    continue
                one_notice_dict = {}
                a = li.find('a')
                ti = li.find('time')
                one_notice_dict['title'] = a.string
                if '[' not in a.string:
                    #题目中没有[的情况
                    one_notice_dict['reporter'] = li.text.split('[')[1].split(']')[0]
                else:
                    #题目中有[的情况
                    one_notice_dict['reporter'] = li.text.split('[')[1+len(re.findall('\[', a.string))].split(']')[0]
                one_notice_dict['time'] = int(datetime.datetime.strptime(ti.string, "%Y.%m.%d").timestamp())
                detail_url = a['href']
                if 'http://' not in detail_url and 'https://' not in detail_url:
                    detail_url = self.root_url + detail_url
                detail_page = requests.get(detail_url)
                detail_bs = BeautifulSoup(detail_page.text, 'lxml')
                one_notice_dict['content'] = '\n'.join(detail_bs.text.split())
                one_notice_dict['url'] = detail_url
                data_list.append(one_notice_dict)
            nextpage_node = page_bs.find(name="a", string=re.compile(".*>.*"))
            if nextpage_node is None or nextpage_node.get('href', None) is None:
                break
            nextpage_url = nextpage_node['href']
            nextpage_url = self.root_url + nextpage_url
            page = requests.get(nextpage_url)
            count += 1
            if page_count > 0 and count >= page_count:
                break
        return pd.DataFrame.from_records(data_list)

    def get_important_announcement_data(self):
        """
        读取重要公告数据，只有一页
        Parameter:
            
        Return:
            pd.DataFrame
        """
        page = requests.get(self.root_url + self.enter_url_important)
        data_list = []
        page_bs = BeautifulSoup(page.text, 'lxml')
        trs_page = page_bs.find_all(name='tr')
        for tr in trs_page:
            one_notice_dict = {}
            a = tr.find('a')
            one_notice_dict['title'] = a.string
            if '[' not in a.string:
                #title中没有[的情况
                one_notice_dict['reporter'] = tr.text.split('[')[1].split(']')[0]
            else:
                #title中有[的情况
                one_notice_dict['reporter'] = tr.text.split('[')[1+len(re.findall('\[', a.string))].split(']')[0]
            detail_url = a['href']
            if 'http://' not in detail_url and 'https://' not in detail_url:
                detail_url = self.root_url + detail_url
            detail_page = requests.get(detail_url)
            detail_bs = BeautifulSoup(detail_page.text, 'lxml')
            tis = re.findall(".*[0-9]*年[0-9]*月[0-9]*日.*", detail_bs.text)
            if len(tis) > 0:
                one_notice_dict['time'] = int(datetime.datetime.strptime(tis[-1].strip(), "%Y年%m月%d日").timestamp())
            else:
                one_notice_dict['time'] = -1
            one_notice_dict['content'] = '\n'.join(detail_bs.text.split())
            one_notice_dict['url'] = detail_url
            data_list.append(one_notice_dict)
        return pd.DataFrame.from_records(data_list)

    

In [118]:
announcementsdata = AnnouncementsData()

In [97]:
eduadmin = announcementsdata.get_eduadmin_announcement_data(2)
print(eduadmin.head(10))

                                   title  reporter        time  \
0    关于举办2020年清华大学离退休教职工八十寿辰祝寿会暨金婚庆典的通知     离退休工作处  1602432000   
1       2020年秋季学期 “第一次·从‘零’开始”大型无偿献血专场通知       校团委  1602432000   
2                         汇款确认通知20201012       财务处  1602432000   
3                中央纪委国家监委通报11起“四风”问题典型案例  纪委办公室监察室  1602432000   
4  清办发(2020) 第32号 关于2020年10月毕业生离校工作安排的通知      党办校办  1602432000   
5         转发教育部关于做好《保障中小企业款项支付条例》施行工作的通知       财务处  1602432000   
6        清华大学博士生“学习与发展”调查邀请（博士二年级、三年级同学）      研究生院  1602432000   
7            2020-2021学年秋第5周校级科研条件平台培训通知      实验室处  1602259200   
8                 关于网站建设运营服务合作供应商遴选结果的通知  信息化工作办公室  1602259200   
9                 关于做好2020年度某人才计划推荐工作的通知       人事处  1602259200   

                                             content  \
0  清华大学\n当前位置：\n首页\n>>\n办公通知\n关于举办2020年清华大学离退休教职工...   
1  清华大学\n当前位置：\n首页\n>>\n办公通知\n2020年秋季学期\n“第一次·从‘零...   
2  清华大学\n当前位置：\n汇款确认\n汇款确认通知20201012\n【发布时间:2020-...   
3  清华大学\n当前位置：\n通知公告\n中央纪委国家监委通报11起“四风”问题典型案例\n【发

AttributeError: 'AnnouncementsData' object has no attribute 'get_important_announcement_data'

In [119]:
important = announcementsdata.get_important_announcement_data()
print(important.head(5))

                                 title  reporter        time  \
0                       审计公告（2020年第7号）  纪委办公室监察室  1602172800   
1    清校发(2020) 第51号 关于开展“小金库”自查自纠工作的通知  纪委办公室监察室  1601222400   
2  关于调整2020～2021学年度秋季学期校历及部分节假日放假调课的通知  纪委办公室监察室  1599753600   
3             2020年秋季学期节假日放假调课安排及调课示意图  纪委办公室监察室  1599753600   
4        关于2020-2021学年度秋季学期教学工作基本要求的通知  纪委办公室监察室  1597852800   

                                             content  \
0  清华大学\n当前位置：\n首页\n>>\n通知通告\n审计公告（2020年第7号）\n202...   
1  清华大学两办信息办公网\n(2020)\n第51号\n关于开展“小金库”自查自纠工作的通知\...   
2  清华大学两办信息办公网\n清办发(2020)\n第29号\n关于调整2020～2021学年度...   
3  教务处办公信息系统\n2020年秋季学期节假日放假调课安排及调课示意图\n【分类:\n综合办...   
4  教务处办公信息系统\n关于2020-2021学年度秋季学期教学工作基本要求的通知\n【分类:...   

                                                 url  
0  http://bmxxfb.cic.tsinghua.edu.cn/docinfo/boar...  
1  http://xxbg.cic.tsinghua.edu.cn/oath/detail.js...  
2  http://xxbg.cic.tsinghua.edu.cn/oath/detail.js...  
3  http://jwcbg.cic.tsinghua.edu.cn/jwcbg/detail_...  
4  

In [122]:
important

,title,reporter,time,content,url
0,审计公告（2020年第7号）,纪委办公室监察室,1602172800,清华大学\n当前位置：\n首页\n>>\n通知通告\n审计公告（2020年第7号）\n202...,http://bmxxfb.cic.tsinghua.edu.cn/docinfo/boar...
1,清校发(2020) 第51号 关于开展“小金库”自查自纠工作的通知,纪委办公室监察室,1601222400,清华大学两办信息办公网\n(2020)\n第51号\n关于开展“小金库”自查自纠工作的通知\...,http://xxbg.cic.tsinghua.edu.cn/oath/detail.js...
2,关于调整2020～2021学年度秋季学期校历及部分节假日放假调课的通知,纪委办公室监察室,1599753600,清华大学两办信息办公网\n清办发(2020)\n第29号\n关于调整2020～2021学年度...,http://xxbg.cic.tsinghua.edu.cn/oath/detail.js...
3,2020年秋季学期节假日放假调课安排及调课示意图,纪委办公室监察室,1599753600,教务处办公信息系统\n2020年秋季学期节假日放假调课安排及调课示意图\n【分类:\n综合办...,http://jwcbg.cic.tsinghua.edu.cn/jwcbg/detail_...
4,关于2020-2021学年度秋季学期教学工作基本要求的通知,纪委办公室监察室,1597852800,教务处办公信息系统\n关于2020-2021学年度秋季学期教学工作基本要求的通知\n【分类:...,http://jwcbg.cic.tsinghua.edu.cn/jwcbg/detail_...
5,清华大学2020年8月授予博士、硕士学位情况通报,纪委办公室监察室,-1,%PDF-1.7\n%����\n1\n0\nobj\n<>/Metadata\n12441...,http://info.tsinghua.edu.cn/html/view/htm/2020...
6,关于2021-2023年改善基本办学条件项目申报工作安排的补充通知,纪委办公室监察室,1593878400,清华大学\n当前位置：\n首页\n>>\n办公通知\n关于2021-2023年改善基本办学条...,http://bmxxfb.cic.tsinghua.edu.cn/boarddetail_...
7,清华大学2020年6月授予博士、硕士学位情况通报,纪委办公室监察室,-1,%PDF-1.7\n%����\n1\n0\nobj\n<>/Metadata\n40061...,http://info.tsinghua.edu.cn/html/view/htm/2020...
8,关于开展经费收支情况自查自纠工作的通知,纪委办公室监察室,1591804800,清华大学两办信息办公网\n清办发(2020)\n第20号\n关于开展经费收支情况自查自纠工作...,http://xxbg.cic.tsinghua.edu.cn/oath/detail.js...
9,清华大学第十八次科研工作讨论会工作方案,纪委办公室监察室,-1,%PDF-1.5\n%����\n1\n0\nobj\n<>>>\nendobj\n2\n0...,http://info.tsinghua.edu.cn/html/view/htm/18th...


In [54]:
root = requests.get("http://postinfo.tsinghua.edu.cn/f/zhongyaogonggao/more")
root_bs = BeautifulSoup(root.text, 'lxml')

In [114]:
root_bs.find_all('tr')[0].find('a')

<a href="http://bmxxfb.cic.tsinghua.edu.cn/docinfo/board/boarddetail.jsp?columnId=sjs07&amp;parentColumnId=sjs&amp;itemSeq=125819" target="_blank" title="审计公告（2020年第7号）">审计公告（2020年第7号）</a>

In [50]:
root_bs.find(name='a', string=re.compile('.*下.*.[页].*'))

In [64]:
detail_url = root_bs.find_all(name='tr')[0].find('a')['href']

In [66]:
detail_page = requests.get(detail_url)
detail_bs = BeautifulSoup(detail_page.text, 'lxml')

In [85]:
datetime.datetime.strptime(re.findall(".*[0-9]*年[0-9]*月[0-9]*日.*", detail_bs.text)[-1].strip(), "%Y年%m月%d日")

datetime.datetime(2020, 10, 9, 0, 0)

In [3]:
root_url = "http://postinfo.tsinghua.edu.cn"
enter_url = "/f/bangongtongzhi/more"
root = requests.get(root_url + enter_url)
root_bs = BeautifulSoup(root.text, 'lxml')

In [4]:
lis_root=[x for x in root_bs.find_all(name='li') if x.find('em') is not None]

In [20]:
li=lis_root[1]

In [30]:
lis_root[0].text.split('[')[1].split(']')[0]

'财务处'

In [32]:
re.findall('\[', "中央纪委国家监委通报11起“四风”问题典型案例</a>  [纪委办公室监察室]	")

['[']

In [29]:
li.find()

<li>
<em class="red fb mr5"></em>
<a href="http://bmxxfb.cic.tsinghua.edu.cn/boarddetail_cat.jsp?columnId=02803&amp;itemSeq=125992" target="_blank">中央纪委国家监委通报11起“四风”问题典型案例</a>  [纪委办公室监察室]	<!--em class="list_ico_new ml10"></em-->
<time datetime="2020.10.12">2020.10.12</time>
</li>

In [6]:
a = li.find('a')
ti = li.find('time')

In [41]:
detail_url = a['href']
if 'http://' not in detail_url and 'https://' not in detail_url:
    detail_url = root_url+detail_url

In [42]:
detail_url

'http://xxbg.cic.tsinghua.edu.cn/oath/detail.jsp?boardid=2709&seq=140042'

In [43]:
detail_page = requests.get(detail_url)
detail_bs = BeautifulSoup(detail_page.text, 'lxml')

In [44]:
'\n'.join(detail_bs.text.split())

'清华大学两办信息办公网\n清办发(2020)\n第32号\n关于2020年10月毕业生离校工作安排的通知\n发布时间：2020-10-12\n访问计数：\n611\n【关闭】\n关于2020年10月毕业生离校工作安排的通知\n清办发〔2020〕32号\n各单位：\n我校2020年10月毕业生离校工作安排如下：\n时\n间\n内\n容\n负责部门\n10月16日8:00\n开通“离校事务处理系统”\n（http://info.tsinghua.edu.cn）\n注册中心\n信息化技术中心\n10月21日\n毕业生离校\n2020年10月16日（周五）上午8:00开通“离校事务处理系统”。毕业生直接登录清华大学信息门户（http://info.tsinghua.edu.cn）进行离校信息查询。\n结合疫情防控实际，涉及办理离校手续的各部门已将大部分工作转为线上办理，其中个别手续仍需线下完成，无法返校的毕业生可委托他人办理相关线下离校手续。具体办理说明详见附件。\n各部门和院系可登录“部门应用系统/学生离校事务处理”，进行离校信息查询及数据处理。请相关部门协助毕业生办理好离校手续。\n特此通知。\n附件：2020年10月毕业生离校手续办理说明.pdf\n校长办公室\n2020年10月10日\n【关闭】\nCopyright©\n清华大学信息化技术中心\n建议使用IE5.0以上版本'

In [6]:
cols=['标题', '单位', '时间', '报告地点:', '主办单位:']

In [7]:
s= '  [保卫部]	'

In [21]:
li = lis_root[0]

In [75]:
a=li.find('a')
ti=li.find('time')

In [87]:
temp = re.match(a['href'], 'http.*')

In [88]:
temp is None

True

In [58]:
s.strip().strip('[').strip(']')

'保卫部'

In [77]:
lis_root

[<li>
 <em class="red fb mr5"></em>
 <a href="/node/280440" target="_blank">关于举行合同管理制度宣讲与业务专项培训的通知</a>  [法务办]	<!--em class="list_ico_new ml10"></em-->
 <time datetime="2020.10.09">2020.10.09</time>
 </li>,
 <li>
 <em class="red fb mr5"></em>
 <a href="http://bmxxfb.cic.tsinghua.edu.cn/boarddetail_cat.jsp?columnId=bwb01&amp;itemSeq=125829" target="_blank">关于10月10日熙春路部分路段实行临时交通管理的通知</a>  [保卫部]	<!--em class="list_ico_new ml10"></em-->
 <time datetime="2020.10.09">2020.10.09</time>
 </li>,
 <li>
 <em class="red fb mr5"></em>
 <a href="http://xsc.tsinghua.edu.cn/info/tzhigg_gg/21352" target="_blank">2020年清华大学本科生台湾、港澳学生宝钢奖学金答辩结果公示</a>  [学生处]	<!--em class="list_ico_new ml10"></em-->
 <time datetime="2020.10.09">2020.10.09</time>
 </li>,
 <li>
 <em class="red fb mr5"></em>
 <a href="http://xxbg.cic.tsinghua.edu.cn/oath/detail.jsp?boardid=2709&amp;seq=139995" target="_blank">清办发(2020) 第31号 清华大学关于开展抗击新冠肺炎疫情表彰评选推荐工作的通知</a>  [党办校办]	<!--em class="list_ico_new ml10"></em-->
 <time datetime="2020.10.

In [25]:
data_dict={}
# total_count=0
# import time
for tag_root in trs_root:
#     start=time.time()
    tag_data_list=[]
    lec_node=tag_root.find('a')
    if lec_node is None:
        break
    lec_tag=lec_node.string
    data_dict[lec_tag]=[]
    lec_url=lec_node['href']
    tag_page=requests.get(root+lec_url)
#     count=0
    while True:
        bs_tag=BeautifulSoup(tag_page.text, 'lxml')
        trs_lec=bs_tag.find_all('tr')
        for tr in trs_lec:
            if tr.get('bgcolor', None) is not None and tr.get('class', None) is None:
                detail_url=tr.find_all('td')[1].find('a')['href']
                detail_link=root+detail_url
                detail_page=requests.get(detail_link)
                bs_detail=BeautifulSoup(detail_page.text, 'lxml')
                trs_detail=bs_detail.find_all('tr')
                tr_detail_dict={}
                for tr in trs_detail:
                    tds=tr.find_all('td')
                    if len(tds)>0:
                        node1=tds[0].find('div')
                        if node1 is not None and node1.string is not None:
                            s=node1.string.strip('')
                            s=s.strip()
                            if s in cols:
                                tr_detail_dict[s]=tds[1].string
                                
                            tr_detail_dict['链接']=detail_link
                tag_data_list.append(tr_detail_dict)
#                 #columns: '报告时间','标题','报告人','地点','链接'
#                 cols=tr.find_all('td')
#                 cols[1]=cols[1].find('a')
#                 data_dict[lec_tag].append([col.string for col in cols] + [root+nextpage_url])
        nextpage_node=bs_tag.find('a', string=re.compile('.*下.*页.*'))
        if nextpage_node is None:
            break
        nextpage_url=nextpage_node['href']
        tag_page=requests.get(root+nextpage_url)
#         count+=1
#         if count >= 2:
#             break
#         print(count)
    data_dict[lec_tag]=pd.DataFrame.from_records(tag_data_list)
#     end=time.time()
#     print(lec_tag, end-start)
#     total_count+=1
#     if total_count>=2:
#         break

<li class="pager-next"><a href="/f/bangongtongzhi/more?field_bgtz_fl_tid=All&amp;page=1" title="去下一个页面">下一页 ›</a></li>

In [61]:
nextpage_node=bs_root.find('a', string=re.compile('.*下.*页.*'))

In [64]:
nextpage_node['href']

'/f/bangongtongzhi/more?field_bgtz_fl_tid=All&page=1'